2차적인 전처리를 진행한 코드입니다.

# Global 변수
1. stan_Seq: Seq 방송개수 column을 만드는데 사용됩니다. 같은 방송이 바로 연달아 방송되는 것을 상품명의 유사성으로 판단하는데, 몇% 이상 일때 유사하다고 판단
2. stan_Day: 하루방송 수 column을 만드는데 사용합니다. 같은 방송이 한날에 방송되는 것을 상품명의 유사성으로 판단하는데, 몇% 이상 일때 유사하다고 판단

In [1]:
stan_Seq = 80
stan_Day = 50

# 필요한 데이터들이 합쳐진 데이터 read
1. raw + 수작업 컬럼+ 외부 컬럼 합쳐진 것(preprocess1_test(train).csv)을 read합니다.
2. 필요한 columns의 정보는 col에 있으며, 다음 컬럼이 없을때에 오류가 자동으로 발생합니다.
3. 6시부터 익일2시까지를 같은 날짜로 파악하기 위한 columns을 만들었습니다.

In [2]:
import warnings
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import re, os
warnings.filterwarnings(action='ignore') 
tqdm.pandas()

cols = ["방송일시", "노출(분)", "마더코드", "상품코드", "상품명", "상품군", "판매단가", "취급액", "브랜드", 
        "지역", "기온", "강수량","vs","lcsch","dc10tca","dsnw","icsr","ss","pa","pv","hm","ws",
        "morning_drama", "popular_program", "holiday", "광고 사람", "상품명_띄어쓰기_set"]
train = pd.read_csv("data/preprocess1_train.csv", index_col = 0)[cols]
test = pd.read_csv("data/preprocess1_test.csv", index_col = 0)[cols]

# 시간데이터 정보로 바꾸기 위해 사용
train["방송일시"] = pd.to_datetime(train["방송일시"])
test["방송일시"] = pd.to_datetime(test["방송일시"])

# 6시부터 익일2시까지를 같은 날짜로 파악하기 위한 columns입니다.
train["방송일시_수정"] = train["방송일시"].apply(lambda x: (x - datetime.timedelta(days=1))\
                                       if x.hour in [0,1,2] else x)
test["방송일시_수정"] = test["방송일시"].apply(lambda x: (x - datetime.timedelta(days=1))\
                                       if x.hour in [0,1,2] else x)

# 필요한 columns 추출

## 월, 시간, 요일, 연달아 쉬는날 컬럼
1. 하루방송이 아침6시 ~ 새벽2시까지 구성돼 있기 떄문에, 이 것들을 하루로 보고 처리할 수 있으나 그렇게 하지 않고 새벽0~2시 방송은 다음날로 처리했습니다. 그 이유는 아래와 같습니다.
  - embedding 전략을 선택했기 때문에 이러한 부분이 크게 영향을 줄 것이라 생각하지 않았습니다.
  - 토요일에서 일요일 넘어가는 새벽의 특성이 일요일에서 월요일로 넘어가는 새벽의 특성보다 일요일에 가깝다고 생각했기 때문입니다. 전자 새벽은 일요일과 같은 휴일로 볼 수 있지만, 후자 새벽은 휴일이라고 보기 힘들기 때문입니다.

변수명|설명
--|--
월| 각 방송날짜의 month
시간| 각 방송날짜의 hour
요일| 각 방송날짜의 weekday
연달아 쉬는 날| 토요일, 일요일, 공휴일을 고려해서 첫 번째 쉬는날은 1 그 다음은 2,3 .... (20년 9월 30일[1], 31일[2]..)

In [3]:
train["월"]  = train["방송일시"].apply(lambda x: x.month)
test["월"]  = test["방송일시"].apply(lambda x: x.month)

In [4]:
train["시간"] = train["방송일시"].apply(lambda x: x.hour)
test["시간"] = test["방송일시"].apply(lambda x: x.hour)

In [5]:
day = dict(zip(range(7), ["월","화","수","목","금","토","일"]))
train["방송요일"] = train["방송일시"].apply(lambda x: day[x.weekday()])
test["방송요일"] = test["방송일시"].apply(lambda x: day[x.weekday()])

In [6]:
# 연달아 쉬는날
result = []
for data in [train,test]:
    tmp_main = data[data["방송요일"].isin(["토", "일"]) | data["holiday"].isin(['Public Holiday', 'National holiday', 'Bank Holiday'])]["방송일시"]
    tmp_main = tmp_main.apply(lambda x: x.date())

    tmp = tmp_main.value_counts().sort_index().reset_index()["index"]
    max_index = len(tmp) - 1
    start_date = True
    i = 1
    j = 0
    def change(x):
        global start_date, i, j
        if start_date:
            num, value = x[0], x[1]
            while num + i <= max_index and value + datetime.timedelta(days=i) == tmp[num+i]:
                i += 1
                start_date = False
            j = i
            return j
        else:
            i -= 1
            if i == 1:
                start_date = True
            return j

    tmp.reset_index().apply(change,axis=1)

    tmp = pd.concat([tmp,tmp.reset_index().apply(change,axis=1)] ,axis = 1).set_index("index")
    tmp_main = tmp_main.reset_index().set_index("방송일시")
    tmp_main["연속 휴일"] = tmp
    data["연속 휴일"] = tmp_main.set_index("index")
    data["연속 휴일"] = data["연속 휴일"].fillna(0).astype(int)
    result.append(data)
train,test = result

## 판매량
1. 취급액이 아닌 판매단가를 y변수로 설정했습니다.

In [7]:
train["판매량"] = train["취급액"] / train["판매단가"]

## 노출(분)\_all
1. 노출(분) 변수에 값들이 대표값을 제외하고 nan으로 채워져 있으므로 모두 그 값을 대표값으로 채워넣었습니다.

In [8]:
result = []
for data in [train,test]:
    tmp = None
    def change(x):
        global tmp
        if str(x) == "nan":
            return tmp
        else:
            tmp = x
            return tmp    
    data["노출(분)_all"] = data["노출(분)"].apply(change)
    result.append(data)
train,test = result

## Seq 방송 개수
1. 같은 방송이 연달아 방송되는 것이 판매량에 주는 영향을 파악하기 위해 넣었습니다.
2. 연속적인 방송의 기준은 한방송이 끝나고 10분 이내로 방송이 되는지입니다. 
3. 같은 방송인지 여부는 띄어쓰기로 구분된 상품명 단어들의 list의 유사성을 바탕으로 판단했습니다. 특정 시간대의 방송 상품명의 단어 list에 10개 단어가 존재하는데, 이전 시간대의 단어들과 비교했을때 10개 중 7개가 겹친다면, 70%의 유사성을 갖는 것입니다. stan_Seq을 기준으로 그 유사성 기준을 정했습니다.
4. 여러제품을 동시에 판매하는 경우 모든 이름의 concat을 하나의 제품 이름으로 간주해 처리했습니다.

In [9]:
result = []
for data in [train,test]:
    before = []
    before_time = None
    num = 1
    def change(val):
        time, reveal, x = val
        global before, num, before_time
        per = (np.isin(np.array(list(x)), np.array(list(before))))
        if per.mean() > stan_Seq * 0.01 and before_time != None and\
        before_time + datetime.timedelta(minutes = reveal + 10) >= time :
            num += 1
        else:
            num = 1
        before = x
        before_time = time
        return num

    tmp = data.groupby(["방송일시","노출(분)_all"])["상품명_띄어쓰기_set"].apply(lambda x: set(" ".join(x).split())).reset_index()
    tmp["index"] = tmp["방송일시"]
    tmp = tmp.set_index("방송일시")[["index", "노출(분)_all","상품명_띄어쓰기_set"]]
    tmp = tmp.apply(change, axis=1)
    tmp.name = "Seq 방송 개수"
    data = pd.merge(data,tmp.reset_index(),on=["방송일시"])
    result.append(data)
train,test = result

## 하루동안 동일 상품개수 몇개 방송했는지
1. 같은 방송이 같은 하루에 여러번 방송되는 영향을 파악하기 위해 넣었습니다.
2. 하루의 기준은 6시 ~ 익일 2시 20분 방송이 끝날때 까지입니다.
3. 같은 방송인지 여부는 Seq 방송 개수 columns의 방법과 동일합니다. 일정 유사도를 넘는 경우 가장 유사도가 높은 상품 기준으로 계산했습니다. 다만 유사도는 상호 비교 했을때 max 유사도를 사용했습니다. A 단어 리스트 10개 B 단어 리스트 20개 일때 서로 겹치는 단어가 5개라면, 5/10, 5/20 중 5/10을 사용했습니다.
4. 여러제품을 동시에 판매하는 경우 모든 이름의 concat을 하나의 제품 이름으로 간주해 처리했습니다.

In [10]:
result = []
for data in [train,test]:
    tmp = data.groupby(["방송일시"])["상품명_띄어쓰기_set"].apply(lambda x: set(" ".join(x).split())).reset_index()
    tmp["방송일시_수정"] = tmp["방송일시"].apply(lambda x: (x - datetime.timedelta(days=1))\
                                           if x.hour in [0,1,2] else x)
    tmp["방송일시_date"] = tmp["방송일시_수정"].apply(lambda x: x.date())
    tmp["하루방송 수"] = None
    tmp["index"] = tmp.index
    tmp2 = tmp[["방송일시","하루방송 수"]]
    tmp = tmp[["방송일시", "상품명_띄어쓰기_set","방송일시_date","index"]]
    def cal_sim(x,ref):
        sim1 = (np.isin(np.array(list(x)), np.array(list(ref)))).mean()
        sim2 = (np.isin(np.array(list(ref)), np.array(list(x)))).mean()
        return max(sim1,sim2)
    def change(val):
        global tmp, tmp2
        date_time, x, date, i = val
        if tmp2.loc[i, "하루방송 수"] != None:
            return
        tmp_data = tmp[(tmp["방송일시_date"] == date) & (tmp["방송일시"] >= date_time)]
        index = tmp_data[tmp_data["상품명_띄어쓰기_set"].apply(cal_sim, ref = x) > stan_Day * 0.01].index
        tmp2.loc[index, "하루방송 수"] = list(range(1, len(index) + 1))
    tmp.progress_apply(change, axis = 1)
    data = pd.merge(data,tmp2[["하루방송 수","방송일시"]], on=["방송일시"])
    result.append(data)
train,test = result

100%|█████████████████████████████████████████████████████████████████████████████| 1617/1617 [00:02<00:00, 667.76it/s]


## 동시방송 상품개수
1. 한 방송에 여러 상품이 동시에 판매되면, 판매량이 분산되는 효과를 반영하기 위해 넣었습니다.

In [11]:
result = []
for data in [train,test]:
    data = pd.merge(data,data.groupby(["방송일시"]).size().reset_index(),on=["방송일시"]).rename(columns = {0:"동시방송 상품개수"})
    result.append(data)
train,test = result

## 성별, 일시불/무이자, 광고 사람, 국내생산
1. 남성, 여성을 의미하는 단어가 상품명에 들어갔을때 효과를 반영하기 위해 넣었습니다.
2. 일시불 무이자를 의미하는 단어가 상품명에 들어갔을때 효과를 반영하기 위해 넣었습니다.
3. 국내에서 인지도 있는 연예인 "팽현숙", "이봉원", "숀리"이 이름에 들어갔을 때 효과를 반영하기 위해 넣었습니다.
4. 국내를 의미하는 단어'제주', '국내', '동해안', '완도', '안동', '영광', '영산포', '여수', '흥양', '진도', '강원도', '국내' 등이 들어갔을 때 판매량에 미치는 영향을 파악하기 위해 넣었습니다.

In [12]:
result = []
for data in [train,test]:
    # 성별
    data.loc[data["상품명"].str.contains("남성") | data["상품명"].str.contains("남자"),"성별"] = "남성"
    data.loc[data["상품명"].str.contains("여성") | data["상품명"].str.contains("여자") ,"성별"] = "여성"

    # 무이자, 일시불
    data.loc[data["상품명"].str.contains("일시불"),"일시불/무이자"] = "일시불"
    data.loc[data["상품명"].str.contains("무이자"),"일시불/무이자"] = "무이자"
    data.loc[data["상품명"].str.contains("(일)"),"일시불/무이자"] = "일시불"
    data.loc[data["상품명"].str.contains("(무)"),"일시불/무이자"] = "무이자"

    # 광고사람
    data["광고 사람"] = data["광고 사람"].isin(["팽현숙", "이봉원", "숀리"]).astype(int)

    # 국내제조, 국내산, 국내제작, 국내산, 국내생산 포함 여부
    data.loc[data["상품명"].str.contains("국내"),"국내생산"] = "국내"
    data["국내생산"] = (data["지역"].notna() | data["국내생산"].notna()).astype(int)
    result.append(data)
train,test = result

## 저장

In [13]:
# "상품명_띄어쓰기_set" 모델에 피팅시킬때 사용해야 하는데 indexing의 편의를 위해서 숫자로 저장
# name_code의 index 순서로 숫자부여
tmp = pd.read_csv("data/name_code.csv",index_col = 0)["상품명_띄어쓰기_set"]
dic = dict(zip(tmp.values,tmp.index))
train["상품명_띄어쓰기_set"] = train["상품명_띄어쓰기_set"].map(dic)
test["상품명_띄어쓰기_set"] = test["상품명_띄어쓰기_set"].map(dic)

In [14]:
# 밑에서 활용하기 위해 저장해둠
tmp_name = test["상품명"]

total_col = ['월', '시간', '연속 휴일', '방송요일', '노출(분)_all', '마더코드', '상품코드', '상품군', '브랜드',
             '판매단가', '광고 사람', '국내생산', '기온', '강수량', 'vs', 'lcsch', 'dc10tca','icsr', 'ss', 'pa', 
             'pv', 'hm', 'ws', 'morning_drama','popular_program', '동시방송 상품개수', 'Seq 방송 개수', '성별', 
             '일시불/무이자', "상품명_띄어쓰기_set", "하루방송 수", "방송일시"]
train = train[total_col + ['판매량', '취급액']]
test = test[total_col]
train.to_csv("data/preprocess2_train"+ str(stan_Seq) + "_" + str(stan_Day) +".csv")
test.to_csv("data/preprocess2_test"+ str(stan_Seq) + "_" + str(stan_Day) +".csv")

# 편성 최적화를 위한 csv 파일
1. 편성 최적화위한 정보를 저장함 -> 계산을 간소화하기 위한 중간 저장과정임

In [15]:
tmp = test[["방송일시","노출(분)_all"]]
tmp = tmp[~tmp["방송일시"].duplicated()].reset_index()
before = None
def change(x):
    global before
    day, time = x
    if before != None and before + datetime.timedelta(minutes= time + 10) >= day:
        before = day
        return True
    else:
        before = day
        return False
tmp["연결여부"] = tmp[["방송일시","노출(분)_all"]].apply(change, axis = 1).astype(int)
tmp["동일날짜"] = tmp["방송일시"].apply(lambda x: (x - datetime.timedelta(days=1)) if x.hour in [0,1,2] else x).apply(lambda x: x.day)
tmp.to_csv("data/test_날짜id.csv")

In [16]:
tmp = test.groupby(["방송일시"]).size().reset_index()["방송일시"].reset_index().rename(columns = {"index":"id"})
test = pd.merge(test, tmp,on = ["방송일시"])
tmp = test[["id", "상품명_띄어쓰기_set"]].reset_index()
tmp["index"] = tmp["index"].astype(str)
tmp["상품명_띄어쓰기_set"] = tmp["상품명_띄어쓰기_set"].astype(str)
a = tmp.groupby("id")["index"].apply(lambda x: ",".join(list(x))).to_frame()
b = tmp.groupby("id")["상품명_띄어쓰기_set"].apply(lambda x: ",".join(list(x)))
tmp = pd.concat([a,b], axis=1)
a = tmp["상품명_띄어쓰기_set"].value_counts().reset_index().rename(columns = {"index" : "상품명_띄어쓰기_set","상품명_띄어쓰기_set" : "반복"})
b = tmp[~tmp["상품명_띄어쓰기_set"].duplicated()]
tmp = pd.merge(a,b)

def change(x):
    index = [int(i) for i in x.split(",")]
    return tmp_name[index].str.cat(sep = " // ")
tmp["name"] = tmp["index"].apply(change)
tmp[["name", "반복", "index"]].to_csv("data/test_상품id.csv")